In [1]:
import time
import numpy as np
from math import sin, cos, atan2
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

MAC = '''
ROBOT_NAME = "Pioneer_p3dx" # MAC
LEFT_MOTOR_NAME = ROBOT_NAME + "_leftMotor" # MAC
RIGHT_MOTOR_NAME = ROBOT_NAME + "_rightMotor" # MAC
SENSOR_PATTERN = "Pioneer_p3dx_ultrasonicSensor{}" # MAC
'''
#LINUX = '''
ROBOT_NAME = "PioneerP3DX" # Linux
LEFT_MOTOR_NAME = "/PioneerP3DX/leftMotor" # Linux
RIGHT_MOTOR_NAME = "/PioneerP3DX/rightMotor" #L inux
SENSOR_PATTERN = "/PioneerP3DX/ultrasonicSensor[{}]" # Linux
#'''

L = 0.331      # distância entre rodas
r = 0.09751     # raio da roda

# parametros para calibrar
K_att = 1.0 #força de atração ao goal
K_rep = 0.8 #força de repulsão aos obstáculos
kv = 0.6 #velocidade linear
kw = 1.2 #velocidade angular

# limites e parâmetros do comportamento
d0 = 0.25 # raio da repulsão
v_max = 0.6 #velocidade máxima linear
w_max = 1.5 #velocidade máxima angular

# condição de parada
DIST_GOAL_STOP = 0.3 

def angle_diff(a, b):
    """Retorna diferença angular (a - b) normalizada para [-pi, pi]."""
    d = a - b
    return np.arctan2(np.sin(d), np.cos(d))

def clip(x, a, b):
    return max(a, min(b, x))

def transform_point_matrix(sim, mat4, point3):

    M = np.array(mat4).reshape((3,4))
    p = np.array([point3[0], point3[1], point3[2], 1.0])
    res = M @ p
    return res  # 3-vector

try:
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)
    
    # Frame que representa o Goal
    goal = 'Goal'
    goalFrame = sim.getObject('/' + goal)
    # Posição do Goal (x, y, theta)
    qgoal_xy = sim.getObjectPosition(goalFrame, sim.handle_world)
    qgoal_orientation = sim.getObjectOrientation(goalFrame, sim.handle_world)



    MAC = '''
    robotHandle = sim.getObject('Pioneer_p3dx') # MAC
    l_wheel = sim.getObject('Pioneer_p3dx_leftMotor') # MAC
    r_wheel = sim.getObject('Pioneer_p3dx_rightMotor') # MAC
    sensorHandles = [sim.getObject(f'Pioneer_p3dx_ultrasonicSensor{i}') for i in range(1, 17)] # MAC
    '''
    #LINUX = '''
    robotHandle = sim.getObject('/PioneerP3DX') # Linux
    l_wheel = sim.getObject('/PioneerP3DX/leftMotor') # Linux
    r_wheel = sim.getObject('/PioneerP3DX/rightMotor') # Linux
    sensorHandles = [sim.getObject(f'/PioneerP3DX/ultrasonicSensor[{i}]') for i in range(16)] # Linux
    #'''

    # Configuração inicial (x, y, theta)
    q_xy = sim.getObjectPosition(robotHandle, sim.handle_world)
    q_orientation = sim.getObjectOrientation(robotHandle, sim.handle_world)
    q = np.array([q_xy[0], q_xy[1], q_orientation[2]])
    
    if len(sensorHandles) == 0:
        raise Exception("AVISO: nenhum sensor encontrado. Verifique nomes na cena.")
    
    # ------------------------------------------Forca de repusao no frame do robo ----------------------------------------------------------------------
    def compute_repulsion_force_robot_frame(sensor_handles, k_rep=K_rep, d0_local=d0):
        F_rep = np.zeros(2)
        for sensorHandle in sensor_handles:
            res = sim.readProximitySensor(sensorHandle)
            detected = bool(res[0])
            if not detected:
                continue

            detectedPoint = res[2]
            distance = float(res[1])
            if distance <= 0 or np.isnan(distance) or np.isinf(distance):
                continue
            
            sensor_mat = sim.getObjectMatrix(sensorHandle, sim.handle_world)
            robot_mat  = sim.getObjectMatrix(robotHandle,  sim.handle_world)
            robot_mat_inv = sim.getMatrixInverse(robot_mat)
            rel = sim.multiplyMatrices(robot_mat_inv, sensor_mat)  # 3x4 matrix
            obs_in_robot_3 = sim.multiplyVector(rel, [detectedPoint[0], detectedPoint[1], detectedPoint[2], 1.0])
            obs_in_robot = np.array(obs_in_robot_3)

            if distance <= d0_local:
                dir_vec = -obs_in_robot[:2]
                norm_dir = np.linalg.norm(dir_vec)
                if norm_dir == 0:
                    continue
                dir_vec = dir_vec / norm_dir
                mag = k_rep * (1.0 / distance - 1.0 / d0_local) / (distance**2 + 1e-6)
                F_rep += mag * dir_vec
        return F_rep

    def compute_goal_attraction_force_robot_frame(k_att=K_att):
        robot_pos_world = sim.getObjectPosition(robotHandle, sim.handle_world)
        robot_orientation = sim.getObjectOrientation(robotHandle, sim.handle_world)
        robot_theta = robot_orientation[2]
        rel_goal_world = np.array(qgoal_xy[:2]) - np.array(robot_pos_world[:2])
        # world -> robot rotation
        c, s = cos(robot_theta), sin(robot_theta)
        R_inv = np.array([[c, s], [-s, c]])
        rel_goal_robot = R_inv @ rel_goal_world
        F_att_robot = k_att * rel_goal_robot
        return F_att_robot, robot_pos_world, robot_theta

    #--------------------------------------------Resultante das forcas ------------------------------------------------------------------------------
    def control_from_force_robot_frame(F_total, robot_theta, k_v=kv, k_w=kw, vlim=v_max, wlim=w_max):
        Fx, Fy = F_total[0], F_total[1]
        v = k_v * Fx
        w = k_w * Fy
        v = float(np.clip(v, -vlim, vlim))
        w = float(np.clip(w, -wlim, wlim))
        return v, w

# ---------------------------Inicialização simulação

    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    # Inicia a simulação
    sim.startSimulation()
    sim.step()

    last_sim_time = sim.getSimulationTime()
    while True:
        sim_time = sim.getSimulationTime()
        dt = sim_time - last_sim_time if sim_time - last_sim_time > 0 else 1e-3
        last_sim_time = sim_time

        # ---- calcula forças no frame do robo ----
        F_att_robot, rob_pos_world, robot_theta = compute_goal_attraction_force_robot_frame()
        F_rep_robot = compute_repulsion_force_robot_frame(sensorHandles)
        F_total_robot = F_att_robot + F_rep_robot

        # evita estouros -> mag 1
        normF = np.linalg.norm(F_total_robot)
        if normF > 1e-6:
            F_total_robot = F_total_robot / normF

        # controlar (v,w) a partir de F_total em frame do robo 
        v, w = control_from_force_robot_frame(F_total_robot, robot_theta)

        # converte para velocidades das rodas 
        wr = (2.0 * v + w * L) / (2.0 * r)
        wl = (2.0 * v - w * L) / (2.0 * r)


        sim.setJointTargetVelocity(l_wheel, wl)
        sim.setJointTargetVelocity(r_wheel, wr)

        #  mesma matriz  de antes (Mdir @ [wr, wl])
        Mdir = np.array([
            [r * np.cos(q[2]) / 2.0, r * np.cos(q[2]) / 2.0],
            [r * np.sin(q[2]) / 2.0, r * np.sin(q[2]) / 2.0],
            [r / L, -r / L]
        ])
        u = np.array([wr, wl])
        q = q + (Mdir @ u) * dt

        gt_pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        gt_ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
        gt_xy = np.array(gt_pos[:2])
        gt_theta = gt_ori[2]

        dist_goal = np.linalg.norm(np.array(qgoal_xy[:2]) - gt_xy)

        print(f"[t={sim_time:.2f}] GT_pos={gt_xy.round(3)} θ={np.degrees(gt_theta):.1f}° | odom={q[:2].round(3)} θ={np.degrees(q[2]):.1f}° | dist_goal={dist_goal:.2f} | v={v:.3f} w={w:.3f}")

        if dist_goal <= DIST_GOAL_STOP:
            print("Chegou perto do objetivo — parando.")
            break

        sim.step()


    sim.setJointTargetVelocity(l_wheel, 0.0)
    sim.setJointTargetVelocity(r_wheel, 0.0)
    time.sleep(0.2)
    sim.stopSimulation()
    print("Simulação parada.")


except Exception as e:
    print(f"An error occurred: {e}")

[t=0.05] GT_pos=[-5.573 -5.65 ] θ=0.0° | odom=[-5.575 -5.65 ] θ=0.1° | dist_goal=13.80 | v=0.325 w=1.009
[t=0.10] GT_pos=[-5.566 -5.65 ] θ=-0.0° | odom=[-5.558 -5.65 ] θ=2.9° | dist_goal=13.80 | v=0.325 w=1.009
[t=0.15] GT_pos=[-5.557 -5.65 ] θ=0.2° | odom=[-5.542 -5.649] θ=5.8° | dist_goal=13.79 | v=0.326 w=1.007
[t=0.20] GT_pos=[-5.547 -5.65 ] θ=0.8° | odom=[-5.526 -5.647] θ=8.7° | dist_goal=13.79 | v=0.331 w=1.001
[t=0.25] GT_pos=[-5.536 -5.651] θ=1.7° | odom=[-5.509 -5.645] θ=11.5° | dist_goal=13.78 | v=0.339 w=0.990
[t=0.30] GT_pos=[-5.523 -5.651] θ=3.1° | odom=[-5.492 -5.641] θ=14.3° | dist_goal=13.77 | v=0.350 w=0.974
[t=0.35] GT_pos=[-5.508 -5.651] θ=4.8° | odom=[-5.474 -5.637] θ=17.1° | dist_goal=13.76 | v=0.364 w=0.953
[t=0.40] GT_pos=[-5.491 -5.652] θ=7.1° | odom=[-5.456 -5.631] θ=19.7° | dist_goal=13.76 | v=0.382 w=0.925
[t=0.45] GT_pos=[-5.473 -5.651] θ=9.7° | odom=[-5.437 -5.624] θ=22.3° | dist_goal=13.75 | v=0.402 w=0.890
[t=0.50] GT_pos=[-5.453 -5.65 ] θ=12.3° | odom=[-

KeyboardInterrupt: 

In [ ]:
sim.stopSimulation()